In [67]:
from __future__ import print_function
import os
import numpy as np
from numpy import newaxis
from sklearn import preprocessing
np.random.seed(1337)

import keras
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Input, Flatten
from keras.layers import Dropout
from keras.models import Model,Sequential
from keras.layers import LSTM
import sys

import pandas as pd
import random

# Reading in data

In [108]:
print("Data read started...")
data = pd.read_csv("nback/result1.csv")
data = data.as_matrix()
print ("Data read finished.")

print(data.shape)

Data read started...
Data read finished.
(622, 19)


# Eliminate EEG data

In [109]:
for i in range (1,5):
    data = np.delete(data, 1, 1) 
    
#data = np.delete(data, 1, 1) 
data = data[:,0:7]
data.shape

(622, 7)

In [110]:
data[0]

array(['3back', 1537.7, 796.38, 36.961999999999996, 4.696727272727268,
       8.947000000000001, 16.387999999999998], dtype=object)

In [111]:
zero = 0
one = 0
two = 0
three = 0

for i in range(len(data)):
    if (data[i, 0] == '0back'):
        zero = zero + 1
    if (data[i, 0] == '1back'):
        one = one + 1
    if (data[i, 0] == '2back'):
        two = two + 1
    if (data[i, 0] == '3back'):
        three = three + 1

print('zero:', zero, ' one: ', one, ' two: ', two, ' three: ', three)

zero: 206  one:  130  two:  139  three:  147


# Dictionary for the levels

In [112]:
level=["0back","1back","2back","3back"]
level2int = dict((p, i) for i, p in enumerate(level))
int2level = dict((i, p) for i, p in enumerate(level))

# Z normalize

In [113]:
for i in range (1,7):
    data[:, i] = preprocessing.scale(data[:, i])

/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConve

In [114]:
back0 = np.zeros((zero,6))
back1 = np.zeros((one,6))
back2 = np.zeros((two,6))
back3 = np.zeros((three,6))
j = 0
k = 0
l = 0
m = 0

for i in range(len(data)):
    if (data[i, 0] == '0back'):
        back0[j] = data[i, 1:]
        j = j+1
    if (data[i, 0] == '1back'):
        back1[k] = data[i, 1:]
        k = k+1
    if (data[i, 0] == '2back'):
        back2[l] = data[i, 1:]
        l = l+1
    if (data[i, 0] == '3back'):
        back3[m] = data[i, 1:]
        m = m+1

In [115]:
y_data = data[:, 0]
for i in range(len(data)):
    y_data[i] = level2int[y_data[i]]

# One-hot encoding

In [116]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()
one_hot = ohe.fit_transform(y_data.reshape(-1,1)).toarray()
print(one_hot)

[[ 0.  0.  0.  1.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  1.]
 ..., 
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]]


# Group data for recurrent network

In [117]:
window = 8
null = np.array([0,0,0,0,0,0])

for i in range(window-1):
        back0 = np.vstack([back0, null])
        back1 = np.vstack([back1, null])
        back2 = np.vstack([back2, null])
        back3 = np.vstack([back3, null])
        
for i in range(window-1):
        back0 = np.vstack([null, back0])
        back1 = np.vstack([null, back1])
        back2 = np.vstack([null, back2])
        back3 = np.vstack([null, back3])

In [118]:
data_length = len(back0) + window - 1 + len(back1) + window - 1 + len(back2) + window - 1 + len(back3) + window - 1

x_data = np.zeros((data_length,window, 6))         # final input data for the network
y_one_hot = np.zeros((data_length,4))             # to store one-hot data groupped


for i in range(len(back0)-window):
    for j in range(window):
        x_data[i, j, :] = back0[i+j]
    y_one_hot[i] = [0,0,0,1]
    

index = 0
for i in range(len(back0)-window, len(back0)-window+ len(back1)-window):
    for j in range(window):
        x_data[i, j, :] = back1[index+j]
    y_one_hot[i] = [0,0,1,0]
    index = index+1

index = 0
for i in range(len(back0)-window+ len(back1)-window, len(back0)-window + len(back1)-window + len(back2)-window):
    for j in range(window):
        x_data[i, j, :] = back2[index+j]
    y_one_hot[i] = [0,1,0,0]
        
    index = index+1
    

index = 0
for i in range(len(back0)-window+ len(back1)-window + len(back2)-window, 
                   len(back0)-window + len(back1)-window + len(back2)-window + len(back3)-window):
    for j in range(window):
        x_data[i, j, :] = back3[index+j]
    y_one_hot[i] = [1,0,0,0]
    index = index+1

# Shuffle data

In [119]:
indices = np.arange(x_data.shape[0])
np.random.shuffle(indices)

x_data = x_data[indices]
y_one_hot = y_one_hot[indices]

# Divide into train, validation and test data

In [120]:
len_data = len(x_data)

nb_test = int(len_data*0.15)
nb_validation = int(len_data*0.15)
nb_train = int(len_data*0.7)

end_valid = nb_train+nb_validation

x_train = x_data[0:nb_train]
y_train = y_one_hot[0:nb_train]

x_valid = x_data[nb_train:end_valid]
y_valid = y_one_hot[nb_train:end_valid]

x_test = x_data[end_valid:]
y_test = y_one_hot[end_valid:]

In [121]:
print(x_train.shape, y_train.shape, x_valid.shape, y_valid.shape, x_test.shape, y_test.shape)

(494, 8, 6) (494, 4) (105, 8, 6) (105, 4) (107, 8, 6) (107, 4)


# Build the net

In [122]:
model = Sequential()

earlyStopping=keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, verbose=0, mode='auto')

model.add(LSTM(64, input_dim=6, input_length=window, return_sequences=True))
model.add(Dropout(0.5))
    

model.add(LSTM(32,return_sequences=False))
model.add(Dropout(0.5))

model.add(Dense(output_dim=y_one_hot.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [123]:
model.fit(x_train, y_train, nb_epoch=200, shuffle=True, callbacks=[earlyStopping], 
          validation_data = (x_valid, y_valid), batch_size=50)

Train on 494 samples, validate on 105 samples
Epoch 1/200
494/494 [==============================] - 4s - loss: 1.1921 - acc: 0.4393 - val_loss: 1.1035 - val_acc: 0.5048
Epoch 2/200
494/494 [==============================] - 0s - loss: 1.0741 - acc: 0.4879 - val_loss: 1.0247 - val_acc: 0.4571
Epoch 3/200
494/494 [==============================] - 0s - loss: 1.0131 - acc: 0.4980 - val_loss: 0.9646 - val_acc: 0.5429
Epoch 4/200
494/494 [==============================] - 0s - loss: 0.9646 - acc: 0.5101 - val_loss: 0.9178 - val_acc: 0.5619
Epoch 5/200
494/494 [==============================] - 0s - loss: 0.9032 - acc: 0.5283 - val_loss: 0.8713 - val_acc: 0.5810
Epoch 6/200
494/494 [==============================] - 0s - loss: 0.8453 - acc: 0.5810 - val_loss: 0.8298 - val_acc: 0.5810
Epoch 7/200
494/494 [==============================] - 0s - loss: 0.8005 - acc: 0.6093 - val_loss: 0.8004 - val_acc: 0.6095
Epoch 8/200
494/494 [==============================] - 0s - loss: 0.7731 - acc: 0.6093

In [125]:
model.evaluate(x_test, y_test, batch_size=50)

100/107 [===========================>..] - ETA: 0s

[0.086966711609998595, 0.88785044612171493]

In [126]:
from sklearn.metrics import classification_report,confusion_matrix

y_pred = model.predict_classes(x_test)
print(y_pred)

p=model.predict_proba(x_test)

target_names = ['class 0(0back)', 'class 1(1back)', 'class 2(2back)', 'class3(3back)']
print(classification_report(np.argmax(y_test,axis=1), y_pred,target_names=target_names))
print(confusion_matrix(np.argmax(y_test,axis=1), y_pred))

107/107 [==============================] - 1s     
[1 2 2 0 2 2 0 3 1 3 3 3 3 2 0 3 1 1 3 1 1 2 2 3 3 0 1 3 3 2 3 2 3 1 0 0 0
 3 3 1 3 2 0 3 0 1 3 1 0 3 3 1 3 1 1 1 1 0 3 0 3 0 0 3 0 0 3 1 3 2 1 0 2 3
 3 3 3 2 0 1 0 3 1 3 3 3 3 1 3 1 3 0 3 0 3 3 3 3 0 3 3 0 2 2 2 3 2]
107/107 [==============================] - 0s     
                precision    recall  f1-score   support

class 0(0back)       1.00      0.72      0.84        32
class 1(1back)       0.95      1.00      0.98        21
class 2(2back)       0.88      1.00      0.94        15
 class3(3back)       0.80      0.92      0.86        39

   avg / total       0.90      0.89      0.89       107

[[23  0  0  9]
 [ 0 21  0  0]
 [ 0  0 15  0]
 [ 0  1  2 36]]
